# data process
将APE210数据转换成符合qwen2微调需要的格式

```
{
    "type": "chatml",
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "What is your name?"
        },
        {
            "role": "assistant",
            "content": "My name is Qwen."
        }
    ],
    "source": "self-made"
}
```

In [1]:
import json

In [2]:
train_origin_path = "data/APE210K/data/train.ape.json"
test_origin_path = "data/APE210K/data/test.ape.json"
valid_origin_path = "data/APE210K/data/valid.ape.json"

In [6]:
template = {
    "type": "chatml",
    "messages": [
        {
            "role": "system",
            "content": "你是一位小学数学专家，能够帮助学生解决数学难题，提高数学能力，轻松掌握数学知识。"
        },
        {
            "role": "user",
            "content": "数学题目：{{original_text}}"
        },
        {
            "role": "assistant",
            "content": "答案是：{{ans}}\n计算公式：{{equation}}"
        }
    ],
    "source": "self-made"
}

def populate_template(data_item, template):
    # 将数据填充到模板中
    populated_template = json.loads(json.dumps(template))  # 深拷贝模板
    for message in populated_template['messages']:
        if '{{original_text}}' in message['content']:
            message['content'] = message['content'].replace('{{original_text}}', data_item['original_text'])
        if '{{ans}}' in message['content']:
            message['content'] = message['content'].replace('{{ans}}', data_item['ans'])
        if '{{equation}}' in message['content']:
            message['content'] = message['content'].replace('{{equation}}', data_item['equation'])
    return populated_template


def formate_data(path):
    new_data = []
    with open(path, "r", encoding="UTF-8") as f:
        for line in f:
            line_item = json.loads(line)
            new_item = populate_template(line_item, template)
            new_data.append(new_item)
    return new_data


def save_formate_data(path, samples, n=-1):
    with open(path, 'w', encoding='utf-8') as f:
        if n == -1:
            n = len(samples)
        for sample in samples[:n]:
            f.write(json.dumps(sample, ensure_ascii=False) + '\n')

In [4]:
new_train = formate_data(train_origin_path)
new_test = formate_data(test_origin_path)
new_valid = formate_data(valid_origin_path)

In [9]:
save_formate_data('data/APE210K-qwen2/train.jsonl', new_train)
save_formate_data('data/APE210K-qwen2/test.jsonl', new_test)
save_formate_data('data/APE210K-qwen2/valid.jsonl', new_valid)

In [8]:
save_formate_data('data/APE210K-qwen2-mini/train.jsonl', new_train, n=2000)
save_formate_data('data/APE210K-qwen2-mini/test.jsonl', new_test, n=500)
save_formate_data('data/APE210K-qwen2-mini/valid.jsonl', new_valid, n=500)